# iBridges First Steps

This note book is for the impatient user. We will take you through the main functionality of the iBridges API.

To follow the notebook you need a valid iRODS configuration file installed in your `.irods` directory. If you do not have that please follow our guide [here](01-Setup-and-connect.ipynb).

## Create an iRODS session (connection to iRODS server)

In [2]:
from ibridges.interactive import interactive_auth
session = interactive_auth()

The `interactive_auth` takes the default `.irods/irods_environment.json` to connect to the iRODS server. You can direct the authentication to a different file like this:

In [3]:
from pathlib import Path
session = interactive_auth(irods_env_path=Path("~").expanduser().joinpath(".irods", "another_env_file.json"))

File not found: /Users/staig001/.irods/another_env_file.json


FileNotFoundError: 

### Checking some session parameters

In [4]:
print(session.username)
print(session.default_resc) # the resource to which data will be uploaded
print(session.zone) # default home for iRODS /zone/home/username
print(session.server_version)
print(session.home)

c.staiger@uu.nl
irodsResc
uu
(4, 2, 12)
/uu/home/research-christine


## A word on iRODS paths

On a local file name we have folders/directories and files. In iRODS we call them collections and data objects. In contrast to files, data objects carry system metadata and user defined metadata. Likewise for collections.

`iBridges` supports iRODS paths with an own module `IrodsPath`. In this module all paths will be relative to the `session.home` unless they start with `/`.

There are three ways to create an `IrodsPath` from your home.

In [5]:
from ibridges import IrodsPath
home = IrodsPath(session, session.home)
home1 = IrodsPath(session, "~")
home2 = IrodsPath(session)

print(home)
print(home1)
print(home2)

/uu/home/research-christine
/uu/home/research-christine
/uu/home/research-christine


`IrodsPaths` implement some functionalities comparable to their counterparts in `Pathlib`.

In [7]:
# some stats
print(home)
print(IrodsPath(session, home).exists())
print(home.dataobject_exists())
print(home.collection_exists())
print(home.parent)

# path manipulation
print(home.joinpath("sub_collection"))

# iRODS system metadata
print(f"Size of {home}: {home.size}") # size in bytes

# iRODS operations
new_coll = IrodsPath.create_collection(home.joinpath("new_coll_name"))
new_coll_path = IrodsPath(session, new_coll.path)
print(f"Created: {new_coll_path}")
new_coll_path = new_coll_path.rename("new_coll_name1")
print(f"Moved to: {new_coll_path}")
new_coll_path.remove()

/uu/home/research-christine
True
False
True
/uu/home
/uu/home/research-christine/sub_collection
Size of /uu/home/research-christine: 49888441501
Created: /uu/home/research-christine/new_coll_name
Moved to: /uu/home/research-christine/new_coll_name1


For a full Tutorial please have a look [here](02-iRODS-paths.ipynb).

## Working with data

[Full Tutorial](03-Working-with-data.ipynb) and a tutorial for [Data Synchronisation](06-Data-sync.ipynb)

In [8]:
from pathlib import Path
from ibridges.util import obj_replicas
from ibridges import upload, download

### Upload a file or folder

Create a local file:

In [9]:
# create a local file
local_path = Path.home().joinpath("demofile.txt")
f = open(local_path, "a")
f.write("My content! Super important")
f.close()

Determine the iRODS path the file should be uploaded to.

In [11]:
irods_path = IrodsPath(session, session.home, "new_coll")
if not irods_path.collection_exists():
    coll = IrodsPath.create_collection(irods_path)

We can first check that an upload would change with the `dry-run`:

In [13]:
# upload the file to our collection
from pprint import pprint
ops = upload(local_path, irods_path, dry_run=True, overwrite=True)
ops.print_summary()

Upload files:

/Users/staig001/demofile.txt -> /uu/home/research-christine/new_coll/demofile.txt



The function will only show what really needs to be updated on iRODS based on checksums. Now we can really upload the file. We will use the `overwrite` flag to update all existing data in iRODS.

In [14]:
changes = upload(local_path, irods_path, overwrite=True)
changes.print_summary()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 27.0/27.0 [00:01<00:00, 25.7B/s]

Uploaded: 1
Upload files:

/Users/staig001/demofile.txt -> /uu/home/research-christine/new_coll/demofile.txt



The `ops` dictionary can also be used to log what has been uploaded.

### Retrieve a  data object from iRODS and checking their status

In [15]:
obj_path = irods_path.joinpath("demofile.txt")

We can retrieve the system metadata of the object through its path:

In [16]:
print('data object name\t', obj_path.name)
print('data object path\t', str(obj_path))
print('data object size\t', obj_path.size)
print('data object checksum\t', obj_path.checksum)

data object name	 demofile.txt
data object path	 /uu/home/research-christine/new_coll/demofile.txt
data object size	 27
data object checksum	 sha2:S75BqiOKr5pS7pQokupxImlIkHOzETWyNqaCtutCHP8=


To retrieve the content of a data object or see on which storage resources the data is stored, we need to access the dataobject:

In [18]:
obj = obj_path.dataobject
print('data replicas\t', obj_replicas(obj))

data replicas	 [(0, 'irodsResc', 'sha2:S75BqiOKr5pS7pQokupxImlIkHOzETWyNqaCtutCHP8=', 27, 'good')]


### Open a data object in read or write mode

In [19]:
stream = obj.open('r')
text = stream.read().decode()
stream.close()
print(text)

My content! Super important


### Download a data object or collection

Downloads work similar as the uploads: use the `dry_run` to see what will change and use the `overwrite` flag to update all already existing data.

In [20]:
ipath = IrodsPath(session, irods_path)
print(ipath)

/uu/home/research-christine/new_coll


In [21]:
from pathlib import Path
local_path = Path.home().joinpath("Downloads")
print("Download folder exists", local_path.is_dir())
local_changes = download(ipath, local_path, overwrite=True)
local_changes.print_summary()

Download folder exists True


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 27.0/27.0 [00:00<00:00, 81.5B/s]

Downloaded: 1, Directories created: 1
Create directories:

/Users/staig001/Downloads/new_coll

Download files:

/uu/home/research-christine/new_coll/demofile.txt -> /Users/staig001/Downloads/new_coll/demofile.txt



## User defined metadata of data objects and collections

[Full Tutorial](04-Metadata.ipynb)

In [22]:
# upload a file
local_path = Path.home().joinpath("demofile.txt")
irods_path = IrodsPath(session, session.home, "new_coll")
if not irods_path.collection_exists():
    coll = IrodsPath.create_collection(irods_path)
changes = upload(local_path, irods_path, overwrite = True)
changes.print_summary()

Skipped unchanged: 1



### Retrieve an iRODS object or collection and list its metadata

In [23]:
obj_path = irods_path.joinpath("demofile.txt")
print(obj_path.meta)

### View, add, set and delete metadata

In [24]:
obj_path.meta.add('NewKey', 'NewValue')
obj_path.meta.add('NewKey', 'AnotherValue')
print(obj_path.meta)

 - (key: 'NewKey', value: 'AnotherValue', units: '')
 - (key: 'NewKey', value: 'NewValue', units: '')


In [25]:
obj_path.meta.delete('NewKey', 'NewValue')
print(obj_path.meta)

 - (key: 'NewKey', value: 'AnotherValue', units: '')


We can also set the meta data to a single key, value, units pair. This will remove any other entries with the same key.

In [26]:
obj_path.meta.set("NewKey", "YetAnotherValue")
print(obj_path.meta)

/var/folders/7z/3klc4jw14w9brkxtlk_312gh0000gp/T/ipykernel_60531/4011836448.py:1: DeprecationWarning: The 'set' method is deprecated and will be removed in iBridges 2.0. You can mimick the old behavior with meta.delete('NewKey'); meta.add('NewKey', 'YetAnotherValue', '')
  obj_path.meta.set("NewKey", "YetAnotherValue")


 - (key: 'NewKey', value: 'YetAnotherValue', units: '')


### Accessing metadata 
With the print function you can quickly inspect the metadata of an iRODS collection or object. If you want to extract and do something with the metadata, you can iterate over all metadata entries. We give a small example below where we assume that the metadata contains a key/name *AUTHOR*:

In [ ]:
for md in obj_path.meta:
    print(md.name, "has value", md.value)

## Search for data

How can you retrieve the location of an iRODS collection or data object by its metadata or partial part? To this end we provide the function `search`

In [ ]:
from ibridges.search import search_data, MetaSearch

#all collections and objects with 
result = search_data(session, metadata=MetaSearch(key="NewKey", value="YetAnotherValue"))
print(result)

In the result we find the `IrodsPath` of the found collections and data objects.

If we are unsure about the exact metadata or collection or object name, we can use the wildcard `%`:

In [ ]:
result = search_data(session, session.home, path_pattern="new_coll/%")
print(result)

## Permissions

**Note: In Yoda 1.9 users cannot set permissions any longer.**

[Full Tutorial](05-Data-Sharing.ipynb)

### Accessing the permissions of a data object or collection in iRODS

Objects and collections have permissions attached to them. Permissions, which work like access levels, must be specified per user or group. The basic permissions are `own` (implies reading and writing), `modify object` (editing and reading), and `read object`.

In [ ]:
from ibridges.permissions import Permissions

# select a file to inspect and set permissions on
item_path = IrodsPath(session, session.home) # Path to collection or data object
collection = item_path.collection  # Get the collection, use item_path.dataobject for data objects

# instantiate permissions with that object
perm = Permissions(session, collection)
print(f'Permissions for {item_path}:\n')
print(perm)

### Available permissions on your iRODS server

In [ ]:
perm.available_permissions

### Adding permissions to a collection or data object

In [ ]:
perm.set('modify object', '<username or group name>')
print(perm)

Note that some permission-types have synonyms:

+ read object: 'read', 'read object', 'read_object'
+ modify object: 'write', 'modify object', 'modify_object'

### Removing permissions

In [ ]:
perm.set('null', '<username or group name>')
print(perm)

### Inheritance

Collections have two special permissions level `inherit` and `noinherit`. From the point in time where inheritance in switched on, all newly added subcollections and data objects will inherit their initial permissions from the collection.

In [ ]:
# Retrieve a collection from iRODS
coll_path = session.home
coll = session.irods_session.collections.get(item_path)
coll_perm = Permissions(session, coll)

#Switch inheritance on
coll_perm.set('inherit')
print(coll_perm)

In [ ]:
# Switch inheritance off
coll_perm.set('noinherit')
print(coll_perm)